# Advent of Code 2023

https://adventofcode.com/2023

## Day 1

In [1]:
sample = '1abc2
pqr3stu8vwx
a1b2c3d4e5f
treb7uchet'

"1abc2\npqr3stu8vwx\na1b2c3d4e5f\ntreb7uchet"

In [2]:
def coordinates(haystack)
  digits = haystack.scan(/\d/).to_a
  [digits[0], digits[-1]].join.to_i
end

:coordinates

In [3]:
def checksum(data)
  data.split("\n").map { |line| coordinates(line) }.sum
end

:checksum

In [4]:
puzzle_1 = File.read('2023-d01a.txt')
checksum(puzzle_1)

55621

In [5]:
puzzle_2 = File.read('2023-d01a.txt')

sample_2 = 'two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen'

NUMBERS = {
  'one' => '1',
  'two' => '2',
  'three' => '3',
  'four' => '4',
  'five' => '5',
  'six' => '6',
  'seven' => '7',
  'eight' => '8',
  'nine' => '9'
}

NUMBER_MATCHER = ['\d', *NUMBERS.keys].join('|')
SCANNABLE_NUMBER_MATCHER = Regexp.new("(?=(#{NUMBER_MATCHER}))")

def verbal_checksum(data)
  data.split("\n").map do |line|
    first_digit_match, *_other_matches, last_digit_match = line.scan(SCANNABLE_NUMBER_MATCHER).flatten
    first_digit = NUMBERS[first_digit_match] || first_digit_match
    last_digit = NUMBERS[last_digit_match] || last_digit_match || first_digit
    [first_digit, last_digit].join.to_i
  end.sum
end

# verbal_checksum(sample_2)

verbal_checksum(puzzle_2)

53592

In [6]:
# scan starts looking for the next match after the end of the previous match.
REGULAR_MATCHER = /(\D\D\D)/
'abcde8fgh'.scan(REGULAR_MATCHER)
# [["abc"], ["fgh"]]

[["abc"], ["fgh"]]

In [7]:

# lookahead matches don't consume the characters they match, so they can overlap.
# scan moves forward by one character after each match.
OVERLAPPING_MATCHER = /(?=(\D\D\D))/
'abcde8fgh'.scan(OVERLAPPING_MATCHER)
# [["abc"], ["bcd"], ["cde"], ["fgh"]]

[["abc"], ["bcd"], ["cde"], ["fgh"]]

## Day 2

In [8]:
d2_sample = 'Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green'

def parse(data)
  data.split("\n").map do |game|
    minimums = {}
    number, results = game.split(':')
    game_id = number.match(/\d+/)[0].to_i

    results.split(';').map do |round|
      counts = {}
      round.split(',').map do |hand|
        number, color = hand.strip.split(' ')
        counts[color] = number.to_i
      end
      minimums.merge!(counts) { |_, old, new| [old, new].max }
    end
    [game_id, minimums]
  end
end

def solve(data)
  parse(data).map do |id, game|
    next id if game['red'] <= 12 && game['green'] <= 13 && game['blue'] <= 14
    0
  end
end

# solve(d2_sample)

d2_input = File.read('2023-d02.txt')
solve(d2_input).sum

2685

In [9]:
def solve_2(data)
  parse(data).sum(0) do |_id, game|
    game.values.reduce(&:*)
  end
end

# solve_2(d2_sample)
solve_2(d2_input)

83707

## Day 3

My instinct is to use a bunch of memory to process the entire file in one go, but I'm choosing to process this line by line as though I might need to do this on something like an ever-growing log file.

In [94]:
d3_sample = '2023-d03-sample.txt'
d3_input = '2023-d03.txt'

SYMBOL_MATCHER = /[^.\d]/

def find_codes(pl, fl, nl)
  match_offset = 0
  fl.scan(/\d+/).map do |num|
    start, width = fl.index(num, match_offset) - 1, num.length + 2
    match_offset = start + width
    if "#{pl[start, width]}#{fl[start, width]}#{nl[start, width]}".match(SYMBOL_MATCHER)
      next num.to_i
    end
    0
  end
end

def solve_3(filename)
  prev_line, focus_line, next_line = '', '', ''
  good_codes = []

  File.foreach(filename) do |line|
    padded_line = ".#{line.chomp}."
    prev_line, focus_line, next_line = focus_line, next_line, padded_line
    next if focus_line.empty?
    prev_line = '.' * focus_line.length if prev_line.empty?
    good_codes += find_codes(prev_line, focus_line, next_line)
  end

  # final line
  good_codes += find_codes(focus_line, next_line, '.' * focus_line.length)

  good_codes
end

# find_codes('.#.......', '.35..633.', '.......!.')

# solve_3(d3_sample)
solve_3(d3_input).sum

# d3_problem = [
#   '.....*..#................506..143........375......77.....155...........400.518...64....773...718..797........694....972.603.....*...........',
#   '....479.795...............*..........800...........*.$.......264*636.......@..............&..*...*.......499...............*...5.20.........',
#   '515...................512.484...*....*...=......390...427...................................644.804.........*...@......-..532............28.'
# ]

# find_codes(*d3_problem)

(irb):3: warning: already initialized constant Object::SYMBOL_MATCHER
(irb):3: warning: previous definition of SYMBOL_MATCHER was here


539713

In [184]:
d3_sample = '2023-d03-sample.txt'
d3_input = '2023-d03.txt'

PART_MATCHER = /^\D*\d+\D+\d+\D*$/

def find_parts(*lines)
  parts_found = []

  lines.each do |line|
    if line[3].match(/\D/)
      applesauce = line
      applesauce[3] = 'Z'
      x = applesauce.match(/(\d+)Z/)
      parts_found << x[1].to_i if x
      x = applesauce.match(/Z(\d+)/)
      parts_found << x[1].to_i if x
    else
      x = line.match(/^.{0,2}\D(\d+)/)
      parts_found << x[1].to_i if x
    end
  end

  raise "Should only be two parts. #{lines}" if parts_found.length != 2

  parts_found[0] * parts_found[1]
end

def find_gears(pl, fl, nl)
  gears_found = []
  match_offset = 0

  while index = fl.index('*', match_offset)
    m_start, m_width = index - 1, 3
    p_start, p_width = index - 3, 7
    match_offset = index + 1
    if "#{pl[m_start, m_width]}Z#{fl[m_start, m_width]}Z#{nl[m_start, m_width]}".match(PART_MATCHER)
      gears_found << find_parts(pl[p_start, p_width], fl[p_start, p_width], nl[p_start, p_width])
    end
  end

  gears_found
end

def solve_3b(filename)
  prev_line, focus_line, next_line = '', '', ''
  gear_ratios = []

  File.foreach(filename) do |line|
    padded_line = "...#{line.chomp}..."
    prev_line, focus_line, next_line = focus_line, next_line, padded_line
    next if focus_line.empty?
    prev_line = '.' * focus_line.length if prev_line.empty?
    gear_ratios << find_gears(prev_line, focus_line, next_line)
  end

  # final line
  gear_ratios << find_gears(focus_line, next_line, '.' * focus_line.length)

  gear_ratios
end

# solve_3b(d3_sample).flatten.sum
solve_3b(d3_input).flatten.sum
# find_parts('789..*9', '...*...', '....602')

# find_parts('..217..', '...*...', '222.333')

# '...8...'.match(/^.{0,2}\D(\d+)/)


(irb):3: warning: already initialized constant Object::PART_MATCHER
(irb):3: warning: previous definition of PART_MATCHER was here


84159075

In [142]:
a = '123.456'
a[3] = 'Z'
a

"123Z456"

## Day 4

In [222]:
d4_sample = 'Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11'

# scratchcards = d4_sample.split("\n").map { |line| line.split(':') }.map { |id, cards| [id, cards.split('|').map(&:strip).map(&:split)] }.map { |id, cards| [id, cards[0] & cards[1]] }
# scratchcards = d4_sample.split("\n")
#   .map { |line| line.split(':') }
#   .map { |id, cards| [id, cards.split('|')
#     .map(&:strip)
#     .map(&:split)] }
#   .map { |_id, cards| (cards[0] & cards[1])
#     .length }
#   .sum(0) { |matches| matches == 0 ? 0 : 2 ** (matches - 1) }

def solve_d4a(data)
  data.split("\n")
  .map { |line| line.split(':') }
  .map { |id, cards| [id, cards.split('|')
    .map(&:strip)
    .map(&:split)] }
  .map { |_id, cards| (cards[0] & cards[1])
    .length }
  .sum(0) { |matches| matches == 0 ? 0 : 2 ** (matches - 1) }
end

# solve_d4a(d4_sample)

d4_puzzle = File.read('2023-d04.txt')

# solve_d4a(d4_puzzle)

def solve_d4b(data)
  winners = data.split("\n")
  .map { |line| line.split(':') }
  .map { |id, cards| [id, cards.split('|')
    .map(&:strip)
    .map(&:split)] }
  .map { |_id, cards| (cards[0] & cards[1])
    .length }

  counts = [1] * winners.length

  winners.each_with_index do |winner, index|
    next if winner == 0
    winner.times do |i|
      counts[index + 1 + i] += counts[index]
    end
  end

  counts
end

# solve_d4b(d4_sample).sum
solve_d4b(d4_puzzle).sum


14814534

In [216]:
xyz = [3, 5, 2]

xyz[0].times do |i|
  puts i
end

0
1
2


3